<a href="https://colab.research.google.com/github/sshd911/Kaggle/blob/main/NovozymesEnzymeStabilityPrediction_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret
!pip install pandas-profiling==3.1.0

In [2]:
import pandas as pd
import numpy as np
from pycaret.regression import setup
from pycaret.regression import compare_models
from pycaret.regression import models
from pycaret.regression import create_model
from pycaret.regression import tune_model
from pycaret.regression import plot_model
from pycaret.regression import finalize_model
from pycaret.regression import predict_model

In [ ]:
!pip install bio-embeddings[all]

In [4]:
from bio_embeddings.embed import SeqVecEmbedder

In [5]:
df = pd.read_csv("/content/drive/MyDrive/workdir/train.csv")
updates = pd.read_csv("/content/drive/MyDrive/workdir/train2.csv")

drop_seq_ids = updates[updates["pH"].isnull()]["seq_id"].values
change_seq_ids = updates[updates["pH"].notnull()]["seq_id"].values

# drop
df.drop(df[df.seq_id.isin(drop_seq_ids)].index, inplace=True)
# change
df.loc[df.seq_id.isin(change_seq_ids), ["pH", "tm"]] = updates[
    updates.seq_id.isin(change_seq_ids)
][["pH", "tm"]].values

df.to_csv("/content/drive/MyDrive/workdir/updated_train.csv", index=False)

train_data = pd.read_csv('/content/drive/MyDrive/workdir/updated_train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/workdir/test.csv')

train_data.head()

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5


In [6]:
train_data = train_data.drop('data_source', axis=1)
test_data = test_data.drop('data_source', axis=1)

**convert** mumpy.Array to pandas.dataframe.Seriese

In [7]:
embedder = SeqVecEmbedder()
embedding = embedder.embed(train_data['protein_sequence'])

a = np.add(embedding[0], embedding[1])
c = np.zeros(len(a))

for i in range(len(a)):
  b = 0
  for j in range(len(a[0]+1)):
    b += a[i][j]
  c[i] = b

train_data['protein_sequence'] = pd.Series(c)

In [8]:
test_embedding = embedder.embed(test_data['protein_sequence'])

t_a = np.add(test_embedding[0], test_embedding[1])
t_c = np.zeros(len(t_a))

for i in range(len(t_a)):
  t_b = 0
  for j in range(len(t_a[0]+1)):
    t_b += t_a[i][j]
  t_c[i] = t_b

test_data['protein_sequence'] = pd.Series(t_c)

In [9]:
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

***setup***

In [10]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2413 entries, 0 to 2412
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seq_id            2413 non-null   int64  
 1   protein_sequence  2413 non-null   float64
 2   pH                2413 non-null   int64  
dtypes: float64(1), int64(2)
memory usage: 75.4 KB


In [11]:
clf = setup(data=a, target="tm", session_id=123, 
            ignore_features=["seq_id"])

INFO:logs:PyCaret Supervised Module
INFO:logs:ML Usecase: regression
INFO:logs:version 2.3.6
INFO:logs:Initializing setup()
INFO:logs:setup(target=tm, ml_usecase=regression, available_plots={'parameter': 'Hyperparameters', 'residuals': 'Residuals', 'error': 'Prediction Error', 'cooks': 'Cooks Distance', 'rfe': 'Feature Selection', 'learning': 'Learning Curve', 'manifold': 'Manifold Learning', 'vc': 'Validation Curve', 'feature': 'Feature Importance', 'feature_all': 'Feature Importance (All)', 'tree': 'Decision Tree', 'residuals_interactive': 'Interactive Residuals'}, train_size=0.7, test_data=None, preprocess=True, imputation_type=simple, iterative_imputation_iters=5, categorical_features=None, categorical_imputation=constant, categorical_iterative_imputer=lightgbm, ordinal_features=None, high_cardinality_features=None, high_cardinality_method=frequency, numeric_features=None, numeric_imputation=mean, numeric_iterative_imputer=lightgbm, date_features=None, ignore_features=['seq_id'], n

TypeError: ignored

**compare model**

In [12]:
best_model = compare_models()
print(best_model)

""


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 2
INFO:logs:[]
INFO:logs:compare_models() succesfully completed......................................


[]


[]


In [14]:
models()

INFO:logs:gpu_param set to False


,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


create **model**

In [13]:
lightgbm = create_model("lightgbm")
print(lightgbm)

INFO:logs:Initializing create_model()
INFO:logs:create_model(estimator=lightgbm, fold=None, round=4, cross_validation=True, predict=True, fit_kwargs=None, groups=None, refit=True, verbose=True, system=True, metrics=None, experiment_custom_tags=None, add_to_model_list=True, probability_threshold=None, display=None, return_train_score=False, kwargs={})
INFO:logs:Checking exceptions
INFO:logs:Preparing display monitor


IntProgress(value=0, description='Processing: ', max=4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:40:00
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Fold,MAE,MSE,RMSE,R2,RMSLE,MAPE


INFO:logs:Importing libraries
INFO:logs:Copying training dataset
INFO:logs:Defining folds
INFO:logs:Declaring metric variables
INFO:logs:Importing untrained model
INFO:logs:Light Gradient Boosting Machine Imported succesfully
INFO:logs:Starting cross validation
INFO:logs:Cross validating with KFold(n_splits=10, random_state=None, shuffle=False), n_jobs=-1


ValueError: ignored

**tuning**

In [ ]:
tuned_lightgbm = tune_model(lightgbm)
print(tuned_lightgbm)

**plot**

In [ ]:
plot_model(tuned_lightgbm, plot="feature")

**predict**

In [ ]:
final_lightgbm = finalize_model(lightgbm)
print(final_lightgbm)

In [ ]:
test_pred = predict_model(final_lightgbm, data=test_data)
test_pred.head()

**make csv**

In [ ]:
subm_data = test_pred[["seq_id", "Label"]]
subm_data = subm_data.rename(columns={"Label" : "tm"})
subm_data.to_csv("submittion.csv", index=False)
subm_data